In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression

import time

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import scipy as sp

import os

In [2]:
PATH_FOLDER = os.path.join(os.getcwd(), "")

In [3]:
df=pd.read_csv("feature_eng_data.csv")

In [4]:
df.set_index('Index', inplace=True)

In [5]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [6]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [7]:
df['diff_productos'] = df.groupby('pk_cid')['Ahorro'].diff()

In [8]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [9]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [10]:
df['compra'].value_counts()

compra
0    5893633
1      43000
Name: count, dtype: int64

In [11]:
df[df['Ahorro'] > 1]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
742,1051788,2018-01-28,2015-08-14,KFC,1.0,2,0,0,0,0,...,37.0,1,39,0,184968.30,1,2,3,NaN,0
839,1052102,2018-01-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,NaN,0
1158,1051365,2018-01-28,2015-08-13,KFC,1.0,3,0,0,0,1,...,8.0,1,29,0,124656.36,2,3,3,NaN,0
2583,1056056,2018-01-28,2015-09-05,KFC,1.0,1,0,0,0,1,...,28.0,1,56,0,51062.61,3,2,2,NaN,0
2859,1055240,2018-01-28,2015-09-03,KAT,1.0,2,0,0,0,1,...,28.0,1,49,0,453427.35,0,2,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954955,1158158,2019-05-28,2016-08-06,KFC,1.0,2,0,0,0,1,...,15.0,0,48,0,108019.92,0,2,1,0.0,0
5956541,1171165,2019-05-28,2016-08-29,KFC,1.0,3,0,0,0,1,...,28.0,1,32,0,160338.78,1,2,2,0.0,0
5957646,1173075,2019-05-28,2016-09-05,KAT,1.0,1,0,0,0,1,...,28.0,1,63,0,212865.30,0,2,2,0.0,0


In [12]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 1052102]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
839,1052102,2018-01-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,NaN,0
474177,1052102,2018-02-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,-1.0,0
695096,1052102,2018-03-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
969043,1052102,2018-04-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
981977,1052102,2018-05-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,1.0,1
1257658,1052102,2018-06-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,-1.0,0
1581945,1052102,2018-07-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
2047280,1052102,2018-08-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
2327249,1052102,2018-09-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0


In [13]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [14]:
df['diff_productos'].fillna(value=0, inplace=True)


In [15]:
df.isnull().any().value_counts()

False    32
Name: count, dtype: int64

In [16]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [17]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [18]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [19]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

In [20]:
df['mes'] = df['pk_partition'].dt.to_period('M')


In [21]:
serie_temporal = df.groupby('mes')['compra'].sum()


In [22]:
serie_temporal

mes
2018-01       0
2018-02    2433
2018-03    2305
2018-04    2264
2018-05    2026
2018-06    3125
2018-07    2942
2018-08    2171
2018-09    2836
2018-10    3036
2018-11    2668
2018-12    2987
2019-01    1827
2019-02    4807
2019-03    2492
2019-04    2287
2019-05    2794
Freq: M, Name: compra, dtype: int32

In [23]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [24]:
#Intentaremos predecir cuantos productos de AHORRO venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])


Predicción de compras para el mes 06 del año 2019: 2862.1481132075423


In [25]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*40
Beneficios

#Beneficios estimados para JUNIO 2019 de cproductos de AHORRO = 114.485 € 

114485.9245283017

### Predicción de acierto del modelo

In [26]:
df = df.sort_values(by='pk_partition', ascending=False)


In [27]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad,mes
Index,,,,,,,,,,,,,,,,,,,,,
5962923,1550586,2019-05-28,2019-05-13,KHE,1.0,3,0,0,0,0,...,37,0,126765.57,0,0,1,0.0,0,0,2019-05
5667089,1490900,2019-05-28,2018-10-27,KHN,1.0,2,0,0,0,0,...,48,0,126765.57,0,1,2,0.0,0,7,2019-05
5667091,1490855,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,22,0,126765.57,0,0,1,0.0,0,7,2019-05
5667092,1490854,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,20,0,83582.13,0,0,1,0.0,0,7,2019-05
5667093,1490824,2019-05-28,2018-10-27,KHQ,1.0,3,0,0,0,0,...,23,0,83582.13,0,0,1,0.0,0,7,2019-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159682,1329635,2018-01-28,2017-10-10,KHE,1.0,3,0,0,0,0,...,20,0,136075.68,0,0,1,0.0,0,4,2018-01
159681,1329634,2018-01-28,2017-10-10,KHE,0.0,3,0,0,0,0,...,21,0,82753.14,0,0,1,0.0,0,4,2018-01
159680,1329629,2018-01-28,2017-10-10,KFC,1.0,2,0,0,0,0,...,48,0,76751.52,1,0,1,0.0,0,4,2018-01


In [28]:
df = df.drop(['entry_date','deceased','diff_productos'], axis=1)



In [29]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [30]:
#Generamos una copia para despues en el recomendador

df_rc2 = df.copy()

In [31]:
df = df.drop(['pk_cid','pk_partition'], axis = 1)

In [32]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (4749306, 39)
Tamaño del conjunto de prueba (X): (1187327, 39)
Tamaño del conjunto de entrenamiento (y): (4749306,)
Tamaño del conjunto de prueba (y): (1187327,)


In [35]:
def AUC(model,X,y):
 return roc_auc_score(y,model.predict_proba(X)[:,1])
def cross_validation(model,X,y):
 Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
 return Kfold_metric.mean()

In [36]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [37]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

In [38]:
#evaluate_model(label='XGBoost', 
# model = xgb.XGBClassifier(), 
# X=X_train, y=y_train)

### Recomendador

In [39]:
df_rc=pd.read_csv("clusters_v2.csv",index_col=0)

In [40]:
df_rc2 = df_rc2[df_rc2["pk_partition"] == "2019-05-28"]

In [41]:
df_rc2

,pk_cid,pk_partition,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,entry_channel_KHD,entry_channel_KHE,entry_channel_KHF,entry_channel_KHK,entry_channel_KHL,entry_channel_KHM,entry_channel_KHN,entry_channel_KHO,entry_channel_KHQ,entry_channel_RED
Index,,,,,,,,,,,,,,,,,,,,,
5962923,1550586,2019-05-28,1.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5667089,1490900,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
5667091,1490855,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
5667092,1490854,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
5667093,1490824,2019-05-28,1.0,3,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815330,1089462,2019-05-28,1.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5815334,1089459,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5815331,1089461,2019-05-28,0.0,2,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [42]:
df_rc2 = df_rc2[["pk_cid","salary","age","segment","gender","region_code", "Financiacion","Ahorro","Cuentas","Antiguedad"]]

In [43]:
df_rc3 = df_rc2.merge(df_rc["cluster"], left_on="pk_cid", right_index=True, how="inner")


In [44]:
df_rc3 = df_rc3.set_index("pk_cid")

In [45]:
df_cluster_3 = df_rc3[df_rc3["cluster"] == 3].copy()

In [46]:
df_cluster_3

,salary,age,segment,gender,region_code,Financiacion,Ahorro,Cuentas,Antiguedad,cluster
pk_cid,,,,,,,,,,
1490855,126765.57,22,3,0,28.0,0,0,1,7,3
1490854,83582.13,20,3,0,33.0,0,0,1,7,3
1490824,83582.13,23,3,0,33.0,0,0,1,7,3
1490823,126765.57,21,3,0,28.0,0,0,0,7,3
1490822,65672.31,22,3,1,45.0,0,0,0,7,3
...,...,...,...,...,...,...,...,...,...,...
1089465,146182.56,24,3,1,15.0,0,0,0,44,3
1089463,54250.35,27,3,0,36.0,0,0,1,44,3
1089462,157631.79,24,3,0,36.0,0,0,1,44,3


In [47]:
# eliminamos la columna de cluster del dataset para que no se use en el cálculo de la similitud de coseno.
df_cluster_3.drop("cluster", axis = 1, inplace = True)

In [48]:
df_cluster_3 = df_cluster_3.sample(frac=0.05, random_state=42)

In [49]:
EXECUTE = True

In [50]:
if EXECUTE:
    # normalizamos nuestro dataset
    st = time.time()
    
    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df_cluster_3)
    df_normalized = pd.DataFrame(df_normalized, columns=df_cluster_3.columns)
    df_normalized.index = df_cluster_3.index


    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    sparse_ratings = sp.sparse.csr_matrix(df_normalized.values)

    st = time.time()
    print("Working with user similarity")
    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_normalized.index, columns = df_normalized.index)
    user_sim_df.columns = map(str, user_sim_df.columns)
    user_sim_df.to_parquet("cf_user_similarity_ah.parquet.gzip")

    print("Working with item similarity")
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_normalized.columns, columns = df_normalized.columns)
    item_sim_df.columns = map(str, item_sim_df.columns)
    item_sim_df.to_parquet("cf_item_similarity_ah.parquet.gzip")

    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))

else:
    user_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_user_similarity_ah.parquet.gzip"))
    item_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_item_similarity_ah.parquet.gzip"))

Normalization took 0.0 minutes
Working with user similarity
Working with item similarity
Total time to calculate similarity took 0.22 minutes.


In [51]:
user_sim_df.head(5)

,1184394,1036381,1040647,1475194,1335646,1339179,1321049,1161436,1031160,1029315,...,1036091,1334712,1458971,1432756,1489385,1335124,1420640,1004499,1311036,1445612
pk_cid,,,,,,,,,,,,,,,,,,,,,
1184394,1.000000,0.796862,0.971504,0.741701,0.963796,0.783821,0.930593,0.996697,0.922917,0.769434,...,0.978904,0.906255,0.743827,0.968250,0.899188,0.971009,0.744614,0.908645,0.957372,0.772683
1036381,0.796862,1.000000,0.859784,0.857685,0.667074,0.924866,0.618879,0.821450,0.695603,0.908273,...,0.853824,0.501875,0.859493,0.706824,0.605817,0.729127,0.906343,0.723389,0.743706,0.909980
1040647,0.971504,0.859784,1.000000,0.777385,0.902919,0.805598,0.876019,0.977288,0.857130,0.775258,...,0.998539,0.843497,0.779393,0.912051,0.888291,0.943840,0.760332,0.919277,0.957997,0.798207
1475194,0.741701,0.857685,0.777385,1.000000,0.700746,0.898518,0.709230,0.747219,0.577259,0.768254,...,0.765980,0.489356,0.999411,0.740013,0.599806,0.777952,0.893324,0.540129,0.787692,0.954635
1335646,0.963796,0.667074,0.902919,0.700746,1.000000,0.713252,0.974931,0.939744,0.922602,0.714825,...,0.917019,0.928023,0.703364,0.952462,0.889333,0.953578,0.631389,0.829946,0.956983,0.702113


In [52]:
item_sim_df.head(5)

,salary,age,segment,gender,region_code,Financiacion,Ahorro,Cuentas,Antiguedad
salary,1.000000,0.810230,0.895772,0.615766,0.757988,0.185702,0.129739,0.744496,0.786966
age,0.810230,1.000000,0.878522,0.617854,0.777054,0.244603,0.178521,0.802907,0.859854
segment,0.895772,0.878522,1.000000,0.654200,0.859418,0.180111,0.114027,0.848451,0.869153
gender,0.615766,0.617854,0.654200,1.000000,0.587058,0.152196,0.092290,0.569183,0.576268
region_code,0.757988,0.777054,0.859418,0.587058,1.000000,0.180601,0.120492,0.757547,0.735942


In [53]:
def top_users(user, df):
    '''
    This function prints the top 10 similar users based on cosine similarity.
    '''

    df.columns = map(int, df.columns)

    if user not in df.columns:
        return('No data available on user {}'.format(user))

    print('Most Similar Users:\n')

    sim_users = df.sort_values(by = user, ascending=False).index[1:8599]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:8599]

    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.4f}'.format(user, sim))

    return sim_users, sim_values

In [54]:
user1 = np.random.choice(user_sim_df.index)


In [55]:
[similar_users, similar_values] = top_users(user1, user_sim_df)

Most Similar Users:

User #1190004, Similarity value: 0.9993
User #1183050, Similarity value: 0.9989
User #1156221, Similarity value: 0.9989
User #1156578, Similarity value: 0.9987
User #1156151, Similarity value: 0.9985
User #1156255, Similarity value: 0.9984
User #1215096, Similarity value: 0.9981
User #1204678, Similarity value: 0.9981
User #1149174, Similarity value: 0.9980
User #1220824, Similarity value: 0.9980
User #1182171, Similarity value: 0.9979
User #1178159, Similarity value: 0.9979
User #1144360, Similarity value: 0.9979
User #1157890, Similarity value: 0.9978
User #1204757, Similarity value: 0.9977
User #1145515, Similarity value: 0.9975
User #1180884, Similarity value: 0.9975
User #1178168, Similarity value: 0.9974
User #1179074, Similarity value: 0.9972
User #1152713, Similarity value: 0.9972
User #1152721, Similarity value: 0.9971
User #1210706, Similarity value: 0.9970
User #1201435, Similarity value: 0.9968
User #1187751, Similarity value: 0.9968
User #1158497, Simi

In [56]:
similar_values

[0.9993145964737349,
 0.9988868050321157,
 0.9988568788843979,
 0.9987225599520758,
 0.9985049921060354,
 0.9984266729377971,
 0.998103421270635,
 0.99806951614304,
 0.998034596432081,
 0.9980091426737216,
 0.9979364335816876,
 0.9979364335816876,
 0.99789851728375,
 0.9977867754505086,
 0.9976856680500369,
 0.9975366834721628,
 0.9975094463435901,
 0.9974353915496207,
 0.997173163539586,
 0.9971657838661481,
 0.997114223823423,
 0.9969888333599501,
 0.9968125054453443,
 0.9967738161886145,
 0.9965883054149445,
 0.9964380080605593,
 0.9963171867700765,
 0.9962927116429978,
 0.9962082480760828,
 0.9962082480760828,
 0.9962082480760828,
 0.9962082480760828,
 0.9961177972544464,
 0.9960755079011692,
 0.9960214753742298,
 0.9959577668832991,
 0.9957560690433336,
 0.9957367793884893,
 0.9956140073833102,
 0.995610995632362,
 0.9956080787918574,
 0.9947631955052916,
 0.9947566051191009,
 0.994701204227699,
 0.9946293629121379,
 0.9944554345523724,
 0.9943605823307462,
 0.9943362991346005,
 0

In [57]:
media = np.mean(similar_values[:8599])
media

0.8246259044356244